In [6]:
import pymongo
mongoUri = 'mongodb://{id}:{pw}@{endpoint}/{db}?authSource={db}'
client = pymongo.MongoClient(
    mongoUri.format(
        id="2020sw",
        pw="swswsw",
        endpoint="doctor.iptime.org",
        db="2020sw"
    ))
db = client.get_database()

print("Available collections:")
for c in db.list_collection_names():
    print("\t", c, db.get_collection(c).estimated_document_count())

Available collections:
	 articles 760535


In [7]:
col = db.get_collection("articles")
#col.update_many({"status": { "$exists":False }}, {"$set": {"status": 0}})
col.create_index([("status", pymongo.DESCENDING)])
col.create_index([("keywords", pymongo.DESCENDING)])

'keywords_-1'

In [8]:
from pprint import pprint
from datetime import datetime
import numpy as np

In [9]:
keyword_dict=dict()
count=0
dataSet=col.find({'date':datetime(2020,2,25,0,0)})
for i,doc in enumerate(dataSet):
    for j in doc['keywords']:
        if(j['name'] not in keyword_dict):
            keyword_dict[j['name']]=count
            count+=1
#print(keyword_dict)
key_list=[]
for i in keyword_dict:
    key_list.append(i)
len(key_list)

['국내',
 '코로나',
 '코로나19',
 '19',
 '진자',
 '사망자',
 '급증',
 '가운데',
 '중앙',
 '중앙아시아',
 '아시아',
 '키르',
 '스스',
 '한국',
 '모든',
 '입국',
 '입국자',
 '14',
 '14일간의',
 '간의',
 '격리',
 '조치',
 '리스',
 '리스펙',
 '사디',
 '리예프',
 '보건',
 '보건부',
 '질병',
 '질병예방',
 '예방',
 '진단',
 '진단국',
 '국장',
 '오늘',
 '25',
 '25일',
 '기자',
 '기자회견',
 '회견',
 '중국',
 '일본',
 '2주간',
 '주간',
 '러시아',
 '언론',
 '보도',
 '국가',
 '감염',
 '감염자가',
 '자가',
 '증가',
 '설명',
 '국경',
 '출입국',
 '출입국관리소',
 '관리소',
 '유입',
 '차단',
 '강화',
 '태세',
 '근무',
 '주재',
 '한국대사관',
 '대사관',
 '보건부가',
 '어제',
 '사람',
 '증상',
 '유무',
 '격리조치',
 '예정',
 '수도',
 '비슈케크',
 '슈케크',
 '인근',
 '마나스',
 '나스',
 '공항',
 '근처',
 '과거',
 '군사',
 '군사기지',
 '기지',
 '장소',
 '문제',
 '14일',
 '이후',
 '허용',
 '별도',
 '시설',
 '치료',
 '통보',
 '소개',
 '이번',
 '4개국',
 '개국',
 '제한',
 '확대',
 '세계',
 '세계보건기구',
 '기구',
 '상황',
 '안정적',
 '결정',
 '계속',
 '당국',
 '현재',
 '자국',
 '키르기스스탄에',
 '교민',
 '1천700명이',
 '700',
 '명이',
 '체류',
 '지방',
 '비행기',
 '승객',
 '검역',
 '장쑤성',
 '우리',
 '국민',
 '대규모',
 '강제',
 '발생',
 '오후',
 '항공',
 '349',
 '349편',
 '인천',
 '인천공항',
 '출발',
 '징

In [10]:
def execRound(key_ndarray, lastRank, edgeSum):
    size=len(key_ndarray[0])
    newWeight=lastRank
    DF=0.85
    for i in range(size):
        pr=0.0
        
        for j in range(size):
            if (key_ndarray[i][j]==0): continue
            else :
                pr+=(lastRank[j]/edgeSum[j])

        newWeight[i]=((1-DF)/size)+(DF*pr)
    return newWeight

In [11]:
nodeNum=len(key_list)
key_ndarray=np.zeros((nodeNum,nodeNum))
pageRank=np.full(nodeNum,1/nodeNum)

for i,doc in enumerate(col.find({'date':datetime(2020,2,25,0,0)})):
    for j in doc['keywords']:
        init_name=j['name']
        for k in doc['keywords']:
            comp_name=k['name']
            if(init_name==comp_name):
                continue
            key_ndarray[keyword_dict[init_name]][keyword_dict[comp_name]]=1
            key_ndarray[keyword_dict[comp_name]][keyword_dict[init_name]]=1

edgeSum=key_ndarray[0]

for i in range(nodeNum):
    edgeSum[i]=key_ndarray[i].sum()
# pageRank=execRound(key_ndarray,pageRank)
pageRank=execRound(key_ndarray,pageRank,edgeSum)
print(pageRank.sum())

1.2863299768063916


In [8]:
Rank_list=[]
for i in range(nodeNum):
    Rank_list.append({'keyword':key_list[i],'pageRank':pageRank[i]})
Rank_list

[{'keyword': '국내', 'pageRank': 0.005372790532533298},
 {'keyword': '코로나', 'pageRank': 0.003306426391420452},
 {'keyword': '코로나19', 'pageRank': 0.003196313540880506},
 {'keyword': '19', 'pageRank': 0.0035639837156115533},
 {'keyword': '진자', 'pageRank': 0.0015864816190681442},
 {'keyword': '사망자', 'pageRank': 0.00046441096622991016},
 {'keyword': '급증', 'pageRank': 0.0005001341612702873},
 {'keyword': '가운데', 'pageRank': 0.0014076570712040215},
 {'keyword': '중앙', 'pageRank': 0.0006293707903851679},
 {'keyword': '중앙아시아', 'pageRank': 3.1778298306644373e-05},
 {'keyword': '아시아', 'pageRank': 0.0003546700449052321},
 {'keyword': '키르', 'pageRank': 6.552141529469621e-05},
 {'keyword': '스스', 'pageRank': 4.5797094881707385e-05},
 {'keyword': '한국', 'pageRank': 0.0014507215603718072},
 {'keyword': '모든', 'pageRank': 0.0006540662557644429},
 {'keyword': '입국', 'pageRank': 0.0006491565292370916},
 {'keyword': '입국자', 'pageRank': 0.0001777037574665865},
 {'keyword': '14', 'pageRank': 0.0008647424569652454},

In [9]:
sorted_list=sorted(Rank_list, key=lambda keyword:(keyword['pageRank']),reverse=True)
for i in range(5):
    print(sorted_list[i])

{'keyword': '국내', 'pageRank': 0.005372790532533298}
{'keyword': '19', 'pageRank': 0.0035639837156115533}
{'keyword': '코로나', 'pageRank': 0.003306426391420452}
{'keyword': '코로나19', 'pageRank': 0.003196313540880506}
{'keyword': '뉴스', 'pageRank': 0.0027991551412807373}


In [7]:
pageRank=execRound(key_ndarray,pageRank,edgeSum)
print(pageRank.sum())

1.234941080498631


In [ ]:
sorted_list=sorted(Rank_list, key=lambda keyword:(keyword['pageRank']),reverse=True)
for i in range(5):
    print(sorted_list[i])

In [ ]:
pageRank